# Libraries

In [28]:
from rake_nltk import Rake
import nltk
import pandas as pd
from unidecode import unidecode
import igraph as ig

# Keyword extraction

In [ ]:
df = pd.read_csv('data/titles.tsv', sep='\t', encoding='UTF-8')
df

Text are cleaned before keyword extraction.

In [68]:
df['TITULO'] = df.TITULO.str.lower()
df['TITULO'] = [unidecode(x) for x in df['TITULO']]
df['TITULO'] = df['TITULO'].str.replace('[^a-zA-Z\s.,:]', '', regex=True)
df['TITULO'] = df['TITULO'].str.replace('\s+', ' ', regex=True)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [71]:
rake_nltk_var = Rake(language='english')

In [ ]:
df_en = df.copy()
df_en.shape

In [73]:
df_key_en = pd.DataFrame()

In [ ]:
for title_id in df_en.index:
    
    rake_nltk_var.extract_keywords_from_text(df_en.TITULO[title_id])
    keyword_extracted = pd.DataFrame(rake_nltk_var.get_ranked_phrases_with_scores())
    
    try:
        df_key_en = pd.concat([df_key_en,
                               pd.DataFrame({'document id':df_en.loc[title_id, 'CODIGO OBRA DIALNET'],
                                             'term':keyword_extracted[1],
                                             'score':keyword_extracted[0]
                                            })])
    except:
        pass

English stopwords are removed.

In [ ]:
stop_en = pd.read_csv('data/english.txt')
stop_en['stopwords'] = [unidecode(str(x)) for x in stop_en['stopwords']]
stop_en.shape

In [ ]:
df_key_en = df_key_en[~df_key_en['term'].isin(stop_en['stopwords'].tolist())]
df_key_en

In [ ]:
unique_terms_df = pd.DataFrame({
    'term': df_key_en['term'].unique()
}).reset_index().rename(columns={'index': 'term id'})

unique_terms_df.to_csv('data/terms_all.tsv', sep='\t', encoding='UTF-8', index=False)

In [ ]:
document_term_df = pd.merge(df_key_en, unique_terms_df, on='term', how='left')
document_term_df = document_term_df[['document id', 'term id']]

document_term_df.to_csv('data/doc_terms_all.tsv', sep='\t', encoding='UTF-8', index=False)